# Neural Networks

- 신경망은 torch.nn 패키지를 이용하여 생성할 수 있음
- nn은 모델을 정의하고 미분하는데 autograd를 사용
- nn.Module은 layer와 output을 반환하는 forward(input) 메서드를 포함

- 실습으로는 랜던 노이즈 이미지 데이터를 이용한 간단 CNN 모델을 만드는 것을 목표로 한다.

### 일반적인 학습과정
- 학습 가능한 매개변수를 갖는 신경망을 정의
- 데이터셋 입력 반복
- 입력을 신경망에서 전파
- 손실을 계산
- 변화도를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치를 갱신 (new weight = weight - learning rate * gradient)

## 1. 신경망 정의

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
'''
    * convolution layer 1
    input image shape : (1, 32, 32)
    output image shape : (6, 30, 30)

    * pooling 2d layer 1
    output image shape : (6, 15, 15)

    * convolution layer 2
    output image shape : (6, 13, 13)
    
    * pooling 2d layer 2
    output image shape : (16, 6, 6)
    
    * fully connected layer 1
    input shape : (16 * 6 * 6)
    output shape : (120)
    
    * fully connected layer 2
    input shape : (120)
    output shape : (84)
    
    * fully connected layer 3 (final layer)
    input shape : (84)
    output shape : (10)
'''

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input channel 1, output channel 6, 3 by 3 convolution matrix 사용
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # 아핀 연산 : y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 크기가 제곱수라면 하나의 숫자만을 특정하여 사용 가능
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

In [24]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [25]:
# forward 함수를 정의하면 자동으로 backward 함수는 aurograde를 사용하여 자동으로 정의됨
# 모델의 학습 가능한 매개변수들은 net.parameters() 에 의해 반환됨
params = list(net.parameters())
len_params = len(params)

# print(params)
print(len_params)
print(params[0].size()) # conv1 layer의 weight
print(params[1].size()) # conv1 layer의 bias
print(params[2].size()) # conv2 layer의 weight
print(params[3].size()) # conv2 layer의 bias
# ...

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])


In [32]:
# 임의의 random image를 이용하여 foward 과정 진행
# (nSamples, nChannels, Height, Width)
# 하나의 샘플만 있다면, input.unsqueeze(0)을 사용하여 가상의 차원을 추가한다.
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0416,  0.0619, -0.0422, -0.0964,  0.1031,  0.0582, -0.0790, -0.0201,
         -0.0089,  0.0175]], grad_fn=<AddmmBackward>)


In [37]:
# 모든 매개변수의 변화도 버퍼를 0으로 설정
# 역전파 단계를 실행하기 전 변화도를 0으로 설정
# backward()를 호출할 때마다 변화도가 buffer에 누적됨
net.zero_grad()
out.backward(torch.randn(1, 10))

## 실습 요약
- torch.Tensor >> backward(), autograd 연산을 지원하는 다차원 배열, 또한 tensor에 대한 gradient를 갖고 있음
- nn.Module >> 신경망 모듈, 매개변수를 encapsulation하는 간편한 방법으로 gpu로 이동, 내보내기, 불러오기 등의 작업을 위한 helper를 제공함
- nn.Parameter >> Tensor의 한 종류, Module에 속성으로 할당될 때 자동으로 매개변수로 등록됨
- autograd.Function >> autograd 연산의 전방향과 역방향 정의를 구현, 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor를 생성하고 history를 부호화하는 함수들과 연결하고 있음

## 2. 손실함수

손실함수는 output, target을 입력으로 받아, 두 입력 값의 차이가 얼마나 멀리 떨어져있는지 추정하는 값을 계산

In [40]:
# mean squared error
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1209, grad_fn=<MseLossBackward>)


실습 모델의 연산 그래프 개요를 다음과 같이 정리할 수 있음

In [49]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## 3. 역전파

In [50]:
# 기존 변화도를 없애는 작업 해주어야 함
# 그렇지 않으면 기존의 것에 누적되기 때문

# 변화도를 초기화
net.zero_grad()

print('conv1 bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1 bias.grad after backward')
print(net.conv1.bias.grad)

conv1 bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1 bias.grad after backward
tensor([ 0.0050, -0.0073,  0.0121,  0.0140,  0.0015, -0.0248])


## 4. 가중치 갱신

- SGD 사용 (Stochastic Gradient Descent)
- new weight = weight - learning rate * gradient

In [51]:
# python code 로 간단히 작성해보기
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [52]:
# SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용
# torch.optim을 이용

In [53]:
import torch.optim as optim

# optimizer 생성
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 학습 과정
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()